# Imports

In [1]:
import re
import PyPDF2
import os
import fitz
import pandas as pd
from nltk.corpus import stopwords

# Fonctions

### Obtenir les Keywords : 

#### Fonction pour ouvrir chaque PDF et extraire le texte

In [2]:
def getTextPDF(PDFpath):
    pdfFileObj = open(PDFpath, 'rb') 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

    pageObj = pdfReader.getPage(0)
    pdf = pageObj.extractText()

    for i in range(1, 3):
        pages = pdfReader.getPage(i)
        pdf += pages.extractText()
    pdfFileObj.close()
    return (pdf)

#### Permet d'obtenir un tableau propre des keywords à partir de texte brute avec des characteres non alpha-numériques

In [3]:
def cleanKeywords(keyword):
    keyword = "".join(keyword)
    keyword = re.sub('Keywords', '', keyword)
    keyword = re.sub('Introduction', '', keyword)
    keyword = re.sub('abstract', '', keyword)
    keyword = re.sub('Background', '', keyword)
    keyword = re.sub('Key Words', '', keyword)
    keyword = re.sub('KEY WORDS', '', keyword)
    
    regex = re.compile(r'[0-9a-z_-]+[A-Z]+[0-9a-z_-]+')
    if (regex.findall(keyword)):
        temp = keyword.split(" ")
        for i in range (len(temp)):
            temp[i] = re.sub(r'([A-Z]+[0-9a-z_]+)', r',\1', temp[i])
        keyword = " ".join(temp)
        
    keyword = re.sub(';', ',', keyword)
    keyword = re.sub('[\s,]{2,20}', ',', keyword)
    keyword = keyword.split(',')
    keyword.pop(0)
    return keyword

#### Permet d'extraire les keywords du texte, les fait passer dans la fonction 'cleanKeywords' et retourne un tableau des keywords

In [4]:
def getKeywords(pdf):
    text = re.sub('[^a-zA-Z0-9;,]', ' ', pdf)
    pattern = re.compile("key\s?words.+abstract|key\s?words.+introduction|key\s?words.+background", 
                         re.IGNORECASE)    
    keywords = pattern.findall(text)
    print(keywords)
    clean_keywords = cleanKeywords(keywords)

    
    return clean_keywords

# Main

In [5]:
path = r'C:\Users\afour\PFE_FOURCAUD'
for fileName in os.listdir(path):
    fullName = os.path.join(path, fileName)
    if (not os.path.isfile(fullName) or fileName[-4:] != '.pdf'):
        continue
    else :
        pdf = getTextPDF(fullName)
        print (getKeywords(pdf))

['Keywords   virtual reality ; engineering education ; systematic r eview ; interactive software;  immersive technology; educational technology   Introduction']
['virtual reality', 'engineering education', 'systematic r eview', 'interactive software', 'immersive technology', 'educational technology', '']


Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


['Keywords  Shannon information, Communication channel, Entropy, Mutual information, Correlation, Neuronal encoding Background']
['Shannon information', 'Communication channel', 'Entropy', 'Mutual information', 'Correlation', 'Neuronal encoding ']
['Key Words  HLA incompatibilityGraft versus host directionHost versus graft directionOverall survivalNonrelapse mortalityabstract']
['HLA incompatibility', 'Graft versus host direction', 'Host versus graft direction', 'Overall survival', 'Nonrelapse mortality']
[]
[]


# 2eme prototype

#  ->

In [2]:
def getKeywordIndex(clean_text):
    T1, T2, T3 = ["keyword", "keywords"], ["keys", "key"], ["word", "words"]
    for i in range (len(clean_text)):
        if clean_text[i][4] in T1:

            return clean_text[i][5], i
        
        elif clean_text[i][4] in T2:

            if clean_text[i+1][4] in T3:


                return clean_text[i][5], i
    return -1 , None

In [3]:
def extract_keywords_block(clean_text):
    T = ["keyword", "keywords", "keys", "key", "word", "words"]
    keywords = []
    keyword_block_index, keyword_index = getKeywordIndex(clean_text)

    if keyword_block_index == -1: return keywords
    for j in range (keyword_index, keyword_index+30):
        if j<len(clean_text):
            if clean_text[j][5]==keyword_block_index:
                if clean_text[j][4] not in T and clean_text[j-1][6]!=clean_text[j][6]:
                    keywords.append(",")
                keywords.append(clean_text[j][4])
            else :
                break            
        else :
            break
    if len(keywords) < 3:
        
        for k in range (j,j+30):
            if k<len(clean_text):
                if clean_text[k][5]==keyword_block_index + 1 and k<len(clean_text):
                    
                    keywords.append(clean_text[k][4])
                else :
                    break
            else :
                break
    return keywords
    

In [4]:
def clean_texte(text, stoplist):
    clean_text = []
    for ligne in text:

        if ligne[4] not in stoplist : 

            ligne = list(ligne)
            ligne[4] = re.sub('[^a-zA-Z0-9.,;-]', '', ligne[4]).lower()

            clean_text.append(ligne)
    return clean_text




In [5]:
def get_page_containing_keyword(text_by_page):
    T = ["keyword", "keywords", "keys", "key"]
    for page in range (len(text_by_page)):
        text = [word[4] for word in text_by_page[page]]

        for word in text:
            word = re.sub('[^a-zA-Z0-9,;-]', '', word).lower()
            if word in T:
                return True, page 
  
    return False, None

In [18]:
import pandas as pd
df = pd.read_csv("Keywords.csv")
df

,Keywords
0,software architecture architecture description...
1,software architecture architecture description...
2,software architecture description languages re...
3,architectural description languages security m...
4,software architecture behaviour modelling arch...
...,...
67,vanets challenges applications
68,vanet modeling mobility routing security
69,ad hoc networks sensors things vehicular ad ho...
70,vehicular networks localization data fusion


In [21]:
import os
path = r'F:\PDFS'
count_echec = 0
count_oui = 0
chianteux = []
T = ["keyword", "keywords", "keys", "key"]
list_keywords = []
stoplist = stopwords.words('english')
i=0
target = []
num_cluster = -1
title=[]
varaibale_hyepr_temporaire_quil_faudra_que_je_supprime_plus_tard_a_bon_entendeur_salut_ps_pas_de_ps = 0
for folderName in os.listdir(path):
    num_cluster+=1
    nFolderName = os.path.join(path, folderName)

    print("\n\n\n\n\n",varaibale_hyepr_temporaire_quil_faudra_que_je_supprime_plus_tard_a_bon_entendeur_salut_ps_pas_de_ps,"\n\n\n\n\n")
    varaibale_hyepr_temporaire_quil_faudra_que_je_supprime_plus_tard_a_bon_entendeur_salut_ps_pas_de_ps = 0

    if (not os.path.isdir(nFolderName)):

        continue
    if folderName == "PDF vehicular":
        continue

    else:
        for fileName in os.listdir(nFolderName):            
            fullName = os.path.join(nFolderName, fileName)

            if (not os.path.isfile(fullName) or fileName[-4:] != '.pdf'):


                continue
            else :
                print(fullName)



                doc = fitz.open(fullName)
                text_by_page = [doc[i].get_text("words") for i in range(doc.page_count)]
                contain_keyword, markpage = get_page_containing_keyword(text_by_page) 
                if contain_keyword == True:
                    varaibale_hyepr_temporaire_quil_faudra_que_je_supprime_plus_tard_a_bon_entendeur_salut_ps_pas_de_ps += 1
                    target.append(num_cluster)
                    title.append(fileName)
                    page = doc[markpage]
                    text = page.get_text("words")
                    clean_text = clean_texte(text, stoplist)

                    keywords = extract_keywords_block(clean_text)
                    count_oui += 1
                    print (keywords)
                    list_keywords.append(keywords)
                else :
                    count_echec += 1
                    chianteux.append(fullName)
                print (i)
                i+=1


        
            
                
           
                

for c in chianteux:
    print(c)
print (count_echec, count_oui)






 0 





F:\PDFS\PDF Architectural Description Language\A Balanced HW SW Teaching Approach for Embedded Microprocessors.pdf
['keywords', 'risc;', 'embedded', 'microprocessor;', 'fpgas;', 'adl']
0
F:\PDFS\PDF Architectural Description Language\A Comparative Study of Keyword and.pdf
['keyword', 'semantic', 'based', 'search', 'engine']
1
F:\PDFS\PDF Architectural Description Language\A Distributed Framework for NLP-Based Keyword and Keyphrase Extraction From Web Pages and Documents.pdf
['keyword', 'keyphrase', 'extraction', 'from', 'web', 'pages']
2
F:\PDFS\PDF Architectural Description Language\A FORMAL SEMANTIC FRAMEWORK FOR SADL LANGUAGE.pdf
['keywords', 'adls,', 'properties', 'formal', 'analysis,', 'rewriting', 'logic.', ',', '1', 'introduction', ',', 'nowadays,', 'formal', 'models', 'proposed', 'software', ',', 'systems', 'complex', 'understand,', 'ensure', ',', 'correct', 'analysis.', 'in', 'addition,', 'parts', 'systems', ',', 'reused', 'within', 'another', 'similar', 'system'

['keywords', 'architecture', 'description', 'languages,', 'specification', ',', 'languages,', 'dynamic', 'architectures,', 'mobile', 'architectures,', '-', ',', 'calculus']
32
F:\PDFS\PDF Architectural Description Language\An Architecture Description Language for Dynamic Sensor-Based Applications.pdf
33
F:\PDFS\PDF Architectural Description Language\An Architecture for Semantically Enriched Data Stream Mining.pdf
[]
34
F:\PDFS\PDF Architectural Description Language\AN ARCHITECTURE-CENTRIC APPROACH FOR MANAGING THE EVOLUTION OF EAI SERVICES-ORIENTED ARCHITECTURE.pdf
['keywords', ',', 'eai', 'architecture,', 'evolution,', 'adl,', 'soa,', 'web', 'services,', 'architecture-centric', 'approach.']
35
F:\PDFS\PDF Architectural Description Language\An Aspect-oriented Software Architecture Description Language AO-ADL Based on XYZ.pdf
['keywords', 'aspect-oriented', 'programming;', 'software', 'architecture;', 'xyzadl;', 'aspect-', ',', 'oriented', 'architecture', 'description', 'language', ',',

F:\PDFS\PDF Architectural Description Language\Architectural_Support_for_Managing_Nomad.pdf
['keywords', 'nomadic', 'applications,', 'architecture', 'description', 'languages,', 'interaction', 'specifications,', 'service', 'binding.']
68
F:\PDFS\PDF Architectural Description Language\ARCHITECTURAL_TRANSFORMATION_AND_RE-_ENGINEERING_F.pdf
['keywords', 'web', 'services,', 'service-oriented', 'architecture,', 'architectural', 'transformation,', 'software', 're-engineering,', 'architectural', 'transformation', 'calculus,', 're-engineering', 'methodology.']
69
F:\PDFS\PDF Architectural Description Language\ARCHITECTURE DESIGN AND APPLICATION FOR AN INTELLIGENT DISTRIBUTED NEWS ARCHIVE.pdf
['keywords', 'news', 'archive', 'architecture,', 'multilingual', 'information', 'retrieval']
70
F:\PDFS\PDF Architectural Description Language\Architecture Level Prediction of Software Quality Attributes.pdf
['keywords', ',', 'software', 'architecture,', 'architecture', 'description', 'language,', 'acme,',

['keywords', ',', 'document', 'image', 'classication', '', ',', 'document', 'classiers', '', 'document', 'classication', '', ',', 'document', 'categorization', '', 'document', 'features', '', ',', 'feature', 'representations', '', 'class', 'models', '', 'classication', ',', 'algorithms', '', 'learning', 'mechanisms', '', 'performance']
98
F:\PDFS\PDF Architectural Description Language\Bridging the Gap between IEEE 1471, Architecture Description Languages and.pdf
['keywords', 'software', 'architecture,', 'architecture', ',', 'description,', 'uml,', 'adl,', 'ieee', '1471,', 'sadl,', ',', 'advanced', 'separation', 'concerns,', 'mdsoc,', 'views,', ',', 'viewpoints,', 'concern', 'space.']
99
F:\PDFS\PDF Architectural Description Language\Building a SOA ontology for universities websites.pdf
[]
100
F:\PDFS\PDF Architectural Description Language\Building_aal_services_using_the_mileo_context_server.pdf
['keywords', 'aal', 'service,', 'context', 'server,', 'three', 'layered', 'context', 'model'

F:\PDFS\PDF Architectural Description Language\Effective Exploiting of Hardware Description  Language as a Tool in Teaching the Digital  and Computer Circuits.pdf
['keywords', 'processor', 'design,', 'central', 'process', 'unit,', 'cpu,', 'instruction', 'set,', 'verilog', ',', '1.', 'introduction', ',', 'using', 'integrated', 'circuits', 'computer', 'architecture', 'digital', 'design', 'courses', ',', 'doesnt', 'mean', 'students', 'good', 'understanding', 'components.', 'often,', ',', 'students', 'may', 'good']
135
F:\PDFS\PDF Architectural Description Language\Efficient VLSI architecture for combinational and sequential circuits with the help of serial and parallel techniques.pdf
['keywords', ',', 'linear', 'feedback', 'shift', 'register,', 'counter,', 'xor', 'xnor', ',', 'gate,', 'maximum', 'frequency.', ',', '1.introduction', ',', 'a', 'linear', 'feedback', 'shift', 'register', 'combination', ',', 'series', 'flip', 'flop', 'xor', 'xnor', 'logic', 'gates.', 'its', ',', 'output', 'pse

F:\PDFS\PDF Architectural Description Language\HHFR A new architecture for Mobile Web Services Principles and Implementations.pdf
['keywords', ',', 'mobile', 'computing,', 'web', 'service,', 'stream', ',', 'message,', 'binary', 'representation', ',', '1.', 'introduction', ',', 'mobile', 'computing', 'gives', 'pervasive', 'computing', ',', 'way', 'access', 'information', 'any-time', 'any-', ',', 'portability', 'remote', 'connectivity.', ',', 'and', 'web', 'service', 'technology', 'gives', 'pervasive']
166
F:\PDFS\PDF Architectural Description Language\High-Performance_Hardware_Description_Language_Spe.pdf
['keywords', 'parallel', 'simulation,', 'hardware', 'description', 'languages,', 'design', 'automation', 'standards,', 'vhdl,', ',', 'vhdl-ams,', 'verilog']
167
F:\PDFS\PDF Architectural Description Language\High_Performance_Data_Encryption_based_o.pdf
['keywords', 'image', 'encryption,', 'fpga,', 'pipeline', 'design,', 'aes.', ',', 'i.', 'introduction', ',', 'advanced', 'growth', 'com

['keywords', 'c-its,', 'reference', 'architecture,', 'sysml,', 'architecture', 'modelling']
199
F:\PDFS\PDF Architectural Description Language\Models of requirements for avionics architecture synthesis safety, capacity and security.pdf
['keywords', 'architecture', 'synthesis,', 'formal', 'specification,', 'requirements,', 'safety,', 'secu-', ',', 'rity,', 'capacity,', 'deps.']
200
F:\PDFS\PDF Architectural Description Language\Model_Based_Software_Development_Issues.pdf
['keywords', '-', 'model', 'based', 'software;', 'domain', 'engineering;', 'domain', 'specificity;', 'transformations.']
201
F:\PDFS\PDF Architectural Description Language\Model_checking_a_software_architecture.pdf
202
F:\PDFS\PDF Architectural Description Language\Moving architectural description from under the technology lamppost.pdf
['keywords', 'software', 'architecture;', 'software', 'architecture', 'description', 'languages']
203
F:\PDFS\PDF Architectural Description Language\Multi-View Architecture Description an

F:\PDFS\PDF Architectural Description Language\Seeking Open Educational Resources to Compose Massive Open Online Courses in Engineering Education An Approach Based on Linked Open Data.pdf
['keywords', 'oer,', 'ocw,', 'mooc,', 'linked', 'data,', 'linked', 'oer', 'data,', 'integration,', 'reuse,', ',', 'openness', ',', 'categories', 'l.1.1,', 'l.1.3,', 'l.1.4,', 'l.3.0,', 'l.3.2']
237
F:\PDFS\PDF Architectural Description Language\Self-adaptive Systems Driven by Runtime Models.pdf
['keywords', 'concept', 'keyword', 'synonymous']
238
F:\PDFS\PDF Architectural Description Language\Semantic Interpretation of Architectural Drawings.pdf
['keywords', ',', 'semantic', 'interpretation,', 'drawing', 'understanding']
239
F:\PDFS\PDF Architectural Description Language\SEMANTIC SEARCH WITHIN EARTH OBSERVATION PRODUCTS DATABASE BASED ON AUTOMATIC TAGGING OF IMAGE CONTENT.pdf
['keywords', 'column', 'used', 'store', 'semi-structured', 'data', ',', 'within', 'keyvalue', 'pairs', 'thus', 'providing', 'no

['keywords', 'dynamic', 'software', 'architectures,', 'architectural', 'reflection.']
268
F:\PDFS\PDF Architectural Description Language\The Language Features and Architecture of B Prolog.pdf
['keywords', 'prolog,', 'logic', 'programming', 'system']
269
F:\PDFS\PDF Architectural Description Language\The SAE Architecture Analysis & Design Language (AADL) A Standard for Engineering Performance Critical Systems.pdf
['keywords', 'architecture,', 'computer', 'architecture,', ',', 'model', 'based', 'development,', 'architecture', 'design', ',', 'language,', ',', 'computer', ',', 'system', ',', 'engineering,', ',', 'computer', 'modelling,', 'aadl,', 'architecture', 'analysis', '', ',', 'design', 'language.']
270
F:\PDFS\PDF Architectural Description Language\The_ArchC_Architecture_Description_Langu.pdf
['key', 'words', 'architecture', 'description', 'language;', 'systemc;', 'isa', 'simulator;', ',', 'compiled', 'simulation.']
271
F:\PDFS\PDF Architectural Description Language\The_Influance_of

In [14]:

a = 0
for t in list_keywords:
    if t == []:
        a+=1
print (a)

33


In [15]:
print (len(title))

281


In [10]:
print(len(list_keywords))

279


In [22]:
Title = []
e = []
n = 0
for k, t in zip(list_keywords, title):

 
    k = " ".join(k)

    k = re.sub('keywords|keyword|key|word|words', '', k)
    k = re.sub("[^A-Za-z]+", " ", k)
    k = k.strip()
    k = re.sub(r'\b\w\b', '', k)
    if len(k)>0 and len (k) < 200:    
        if k[0]== " ":
            k = re.sub(r'^\s+', '', k)
        e.append(k)
        Title.append(re.sub(r'\.pdf$', '', t))
    n+=1

y=0    
for i in e :
    print(y)
    print (i)
    y+=1
    
print (len (e))
print (len(Title)) 

0
risc embedded microprocessor fpgas adl
1
semantic based search engine
2
phrase extraction from web pages
3
software architecture architecture description languages definition classification comparison
4
government isp web services adaptive mechanism
5
software architecture architecture description language software environment evolution implementation mapping
6
software architecture maturity models isoiecieee
7
complexity metrics product line architecture evalua tion product line architecture quality attributes
8
ea management methods modeling language business process model no tation
9
sda scl fpga fsm one hot encoding
10
service oriented architecture mobile phones dynamical services invocation
11
software architecture description languages rewriting logic maude contracts
12
architectural description languages security multiagent systems
13
dashboards business processes monitoring controlling xml
14
language engineering nlp architecture uima
15
visual rule based language pervasive c

In [31]:
del e[13]
del Title[13]
print (len (e))
print (len(Title)) 

211
211


In [32]:
stockage_temporaire += e
title_temporaire += Title
print (len(stockage_temporaire))
print (e[0:6])

422
['risc embedded microprocessor fpgas adl', 'semantic based search engine', 'phrase extraction from web pages', 'software architecture architecture description languages definition classification comparison', 'government isp web services adaptive mechanism', 'software architecture architecture description language software environment evolution implementation mapping']


In [33]:
df = pd.DataFrame({'Keywords': stockage_temporaire, 'Title': title_temporaire})

df.to_csv('CSV_PDFS.csv', index = False, encoding='utf-8')
df

,Keywords,Title
0,vehicular ad hoc network vanet data disseminat...,A comprehensive survey on data communication i...
1,blockchain internet things security vehicular ...,A Distributed Blockchain Based Vehicular Netwo...
2,vehicular ad hoc network fast handover authent...,A Fast-Handover-Supported Authentication
3,binary classification machine learning misbeha...,A new procedure for misbehavior detection in v...
4,routing protocol trust vanet vehicular ad hoc ...,A New Trusted Routing Protocol for Vehicular A...
...,...,...
417,systems of systems emergent behaviors model tr...,Validating Emergent Behaviors in Systems-of-Sy...
418,uml proles architectural validation,Validating_UML_models_against_architectu
419,architectural description multiple,Views and Viewpoints in Software Systems Archi...
420,xml web services qos capacity planning,Web Services Planning Concepts


In [11]:
import pandas as pd
df = pd.read_csv("anoted_df.csv")
DF=df.iloc[27:33]
DF

,Keywords,Title,cluster,categorie
27,vanet modeling mobility routing security,Vehicular Ad Hoc Networks (VANETs) An Overvie...,1,vehicular network
28,ad hoc networks sensors things vehicular ad ho...,Vehicular Adhoc Network (VANET)An Introduction,1,vehicular network
29,vehicular networks localization data fusion,Vehicular_ad_hoc_networks_A_new_challeng,1,vehicular network
30,vehicular ad hoc networks simulation wireless,Wireless Communications - 2011 - Martinez - A ...,1,vehicular network
31,software architecture architecture description...,A Framework for Classifying and Comparing Arch...,0,architecture software
32,software architecture architecture description...,A Language and Environment for Architecture-Ba...,0,architecture software
